In [1]:
# Dependencies to Visualize the model
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Filepaths, numpy, and Tensorflow
import os
import numpy as np
import keras

# Preprocessing imports
import glob
import os.path as path
import imageio
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Model Imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score

from datetime import datetime

Using TensorFlow backend.


In [2]:
# root = 'samples'
root = 'Data'

# Load the images
file_paths = []
labels = []
targets = []

emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
x=0
e=0
for emotion in emotions:
    print(emotion)
    for filename in glob.glob(f"{root}/{emotion}/*.jpg"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
        print(x)
        if x == 1000:
            x = 0
            break
            
    for filename in glob.glob(f"{root}/{emotion}/*.png"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
        print(x)
        if x == 1000:
            x=0
            break
    e+=1
    x=0

angry
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [3]:
# Load the images
images = []
image_size = (50,50)

for path in file_paths:
    print(f"{x} of {len(file_paths)}")
    x+=1
    img = load_img(path,target_size = image_size,color_mode='grayscale')
    img = img_to_array(img)
    images.append(img)


0 of 13000
1 of 13000
2 of 13000
3 of 13000
4 of 13000
5 of 13000
6 of 13000
7 of 13000
8 of 13000
9 of 13000
10 of 13000
11 of 13000
12 of 13000
13 of 13000
14 of 13000
15 of 13000
16 of 13000
17 of 13000
18 of 13000
19 of 13000
20 of 13000
21 of 13000
22 of 13000
23 of 13000
24 of 13000
25 of 13000
26 of 13000
27 of 13000
28 of 13000
29 of 13000
30 of 13000
31 of 13000
32 of 13000
33 of 13000
34 of 13000
35 of 13000
36 of 13000
37 of 13000
38 of 13000
39 of 13000
40 of 13000
41 of 13000
42 of 13000
43 of 13000
44 of 13000
45 of 13000
46 of 13000
47 of 13000
48 of 13000
49 of 13000
50 of 13000
51 of 13000
52 of 13000
53 of 13000
54 of 13000
55 of 13000
56 of 13000
57 of 13000
58 of 13000
59 of 13000
60 of 13000
61 of 13000
62 of 13000
63 of 13000
64 of 13000
65 of 13000
66 of 13000
67 of 13000
68 of 13000
69 of 13000
70 of 13000
71 of 13000
72 of 13000
73 of 13000
74 of 13000
75 of 13000
76 of 13000
77 of 13000
78 of 13000
79 of 13000
80 of 13000
81 of 13000
82 of 13000
83 of 13000
84

754 of 13000
755 of 13000
756 of 13000
757 of 13000
758 of 13000
759 of 13000
760 of 13000
761 of 13000
762 of 13000
763 of 13000
764 of 13000
765 of 13000
766 of 13000
767 of 13000
768 of 13000
769 of 13000
770 of 13000
771 of 13000
772 of 13000
773 of 13000
774 of 13000
775 of 13000
776 of 13000
777 of 13000
778 of 13000
779 of 13000
780 of 13000
781 of 13000
782 of 13000
783 of 13000
784 of 13000
785 of 13000
786 of 13000
787 of 13000
788 of 13000
789 of 13000
790 of 13000
791 of 13000
792 of 13000
793 of 13000
794 of 13000
795 of 13000
796 of 13000
797 of 13000
798 of 13000
799 of 13000
800 of 13000
801 of 13000
802 of 13000
803 of 13000
804 of 13000
805 of 13000
806 of 13000
807 of 13000
808 of 13000
809 of 13000
810 of 13000
811 of 13000
812 of 13000
813 of 13000
814 of 13000
815 of 13000
816 of 13000
817 of 13000
818 of 13000
819 of 13000
820 of 13000
821 of 13000
822 of 13000
823 of 13000
824 of 13000
825 of 13000
826 of 13000
827 of 13000
828 of 13000
829 of 13000
830 of 13000

1363 of 13000
1364 of 13000
1365 of 13000
1366 of 13000
1367 of 13000
1368 of 13000
1369 of 13000
1370 of 13000
1371 of 13000
1372 of 13000
1373 of 13000
1374 of 13000
1375 of 13000
1376 of 13000
1377 of 13000
1378 of 13000
1379 of 13000
1380 of 13000
1381 of 13000
1382 of 13000
1383 of 13000
1384 of 13000
1385 of 13000
1386 of 13000
1387 of 13000
1388 of 13000
1389 of 13000
1390 of 13000
1391 of 13000
1392 of 13000
1393 of 13000
1394 of 13000
1395 of 13000
1396 of 13000
1397 of 13000
1398 of 13000
1399 of 13000
1400 of 13000
1401 of 13000
1402 of 13000
1403 of 13000
1404 of 13000
1405 of 13000
1406 of 13000
1407 of 13000
1408 of 13000
1409 of 13000
1410 of 13000
1411 of 13000
1412 of 13000
1413 of 13000
1414 of 13000
1415 of 13000
1416 of 13000
1417 of 13000
1418 of 13000
1419 of 13000
1420 of 13000
1421 of 13000
1422 of 13000
1423 of 13000
1424 of 13000
1425 of 13000
1426 of 13000
1427 of 13000
1428 of 13000
1429 of 13000
1430 of 13000
1431 of 13000
1432 of 13000
1433 of 13000
1434 o

1962 of 13000
1963 of 13000
1964 of 13000
1965 of 13000
1966 of 13000
1967 of 13000
1968 of 13000
1969 of 13000
1970 of 13000
1971 of 13000
1972 of 13000
1973 of 13000
1974 of 13000
1975 of 13000
1976 of 13000
1977 of 13000
1978 of 13000
1979 of 13000
1980 of 13000
1981 of 13000
1982 of 13000
1983 of 13000
1984 of 13000
1985 of 13000
1986 of 13000
1987 of 13000
1988 of 13000
1989 of 13000
1990 of 13000
1991 of 13000
1992 of 13000
1993 of 13000
1994 of 13000
1995 of 13000
1996 of 13000
1997 of 13000
1998 of 13000
1999 of 13000
2000 of 13000
2001 of 13000
2002 of 13000
2003 of 13000
2004 of 13000
2005 of 13000
2006 of 13000
2007 of 13000
2008 of 13000
2009 of 13000
2010 of 13000
2011 of 13000
2012 of 13000
2013 of 13000
2014 of 13000
2015 of 13000
2016 of 13000
2017 of 13000
2018 of 13000
2019 of 13000
2020 of 13000
2021 of 13000
2022 of 13000
2023 of 13000
2024 of 13000
2025 of 13000
2026 of 13000
2027 of 13000
2028 of 13000
2029 of 13000
2030 of 13000
2031 of 13000
2032 of 13000
2033 o

2550 of 13000
2551 of 13000
2552 of 13000
2553 of 13000
2554 of 13000
2555 of 13000
2556 of 13000
2557 of 13000
2558 of 13000
2559 of 13000
2560 of 13000
2561 of 13000
2562 of 13000
2563 of 13000
2564 of 13000
2565 of 13000
2566 of 13000
2567 of 13000
2568 of 13000
2569 of 13000
2570 of 13000
2571 of 13000
2572 of 13000
2573 of 13000
2574 of 13000
2575 of 13000
2576 of 13000
2577 of 13000
2578 of 13000
2579 of 13000
2580 of 13000
2581 of 13000
2582 of 13000
2583 of 13000
2584 of 13000
2585 of 13000
2586 of 13000
2587 of 13000
2588 of 13000
2589 of 13000
2590 of 13000
2591 of 13000
2592 of 13000
2593 of 13000
2594 of 13000
2595 of 13000
2596 of 13000
2597 of 13000
2598 of 13000
2599 of 13000
2600 of 13000
2601 of 13000
2602 of 13000
2603 of 13000
2604 of 13000
2605 of 13000
2606 of 13000
2607 of 13000
2608 of 13000
2609 of 13000
2610 of 13000
2611 of 13000
2612 of 13000
2613 of 13000
2614 of 13000
2615 of 13000
2616 of 13000
2617 of 13000
2618 of 13000
2619 of 13000
2620 of 13000
2621 o

...

10189 of 13000
10190 of 13000
10191 of 13000
10192 of 13000
10193 of 13000
10194 of 13000
10195 of 13000
10196 of 13000
10197 of 13000
10198 of 13000
10199 of 13000
10200 of 13000
10201 of 13000
10202 of 13000
10203 of 13000
10204 of 13000
10205 of 13000
10206 of 13000
10207 of 13000
10208 of 13000
10209 of 13000
10210 of 13000
10211 of 13000
10212 of 13000
10213 of 13000
10214 of 13000
10215 of 13000
10216 of 13000
10217 of 13000
10218 of 13000
10219 of 13000
10220 of 13000
10221 of 13000
10222 of 13000
10223 of 13000
10224 of 13000
10225 of 13000
10226 of 13000
10227 of 13000
10228 of 13000
10229 of 13000
10230 of 13000
10231 of 13000
10232 of 13000
10233 of 13000
10234 of 13000
10235 of 13000
10236 of 13000
10237 of 13000
10238 of 13000
10239 of 13000
10240 of 13000
10241 of 13000
10242 of 13000
10243 of 13000
10244 of 13000
10245 of 13000
10246 of 13000
10247 of 13000
10248 of 13000
10249 of 13000
10250 of 13000
10251 of 13000
10252 of 13000
10253 of 13000
10254 of 13000
10255 of 1

10740 of 13000
10741 of 13000
10742 of 13000
10743 of 13000
10744 of 13000
10745 of 13000
10746 of 13000
10747 of 13000
10748 of 13000
10749 of 13000
10750 of 13000
10751 of 13000
10752 of 13000
10753 of 13000
10754 of 13000
10755 of 13000
10756 of 13000
10757 of 13000
10758 of 13000
10759 of 13000
10760 of 13000
10761 of 13000
10762 of 13000
10763 of 13000
10764 of 13000
10765 of 13000
10766 of 13000
10767 of 13000
10768 of 13000
10769 of 13000
10770 of 13000
10771 of 13000
10772 of 13000
10773 of 13000
10774 of 13000
10775 of 13000
10776 of 13000
10777 of 13000
10778 of 13000
10779 of 13000
10780 of 13000
10781 of 13000
10782 of 13000
10783 of 13000
10784 of 13000
10785 of 13000
10786 of 13000
10787 of 13000
10788 of 13000
10789 of 13000
10790 of 13000
10791 of 13000
10792 of 13000
10793 of 13000
10794 of 13000
10795 of 13000
10796 of 13000
10797 of 13000
10798 of 13000
10799 of 13000
10800 of 13000
10801 of 13000
10802 of 13000
10803 of 13000
10804 of 13000
10805 of 13000
10806 of 1

11682 of 13000
11683 of 13000
11684 of 13000
11685 of 13000
11686 of 13000
11687 of 13000
11688 of 13000
11689 of 13000
11690 of 13000
11691 of 13000
11692 of 13000
11693 of 13000
11694 of 13000
11695 of 13000
11696 of 13000
11697 of 13000
11698 of 13000
11699 of 13000
11700 of 13000
11701 of 13000
11702 of 13000
11703 of 13000
11704 of 13000
11705 of 13000
11706 of 13000
11707 of 13000
11708 of 13000
11709 of 13000
11710 of 13000
11711 of 13000
11712 of 13000
11713 of 13000
11714 of 13000
11715 of 13000
11716 of 13000
11717 of 13000
11718 of 13000
11719 of 13000
11720 of 13000
11721 of 13000
11722 of 13000
11723 of 13000
11724 of 13000
11725 of 13000
11726 of 13000
11727 of 13000
11728 of 13000
11729 of 13000
11730 of 13000
11731 of 13000
11732 of 13000
11733 of 13000
11734 of 13000
11735 of 13000
11736 of 13000
11737 of 13000
11738 of 13000
11739 of 13000
11740 of 13000
11741 of 13000
11742 of 13000
11743 of 13000
11744 of 13000
11745 of 13000
11746 of 13000
11747 of 13000
11748 of 1

12231 of 13000
12232 of 13000
12233 of 13000
12234 of 13000
12235 of 13000
12236 of 13000
12237 of 13000
12238 of 13000
12239 of 13000
12240 of 13000
12241 of 13000
12242 of 13000
12243 of 13000
12244 of 13000
12245 of 13000
12246 of 13000
12247 of 13000
12248 of 13000
12249 of 13000
12250 of 13000
12251 of 13000
12252 of 13000
12253 of 13000
12254 of 13000
12255 of 13000
12256 of 13000
12257 of 13000
12258 of 13000
12259 of 13000
12260 of 13000
12261 of 13000
12262 of 13000
12263 of 13000
12264 of 13000
12265 of 13000
12266 of 13000
12267 of 13000
12268 of 13000
12269 of 13000
12270 of 13000
12271 of 13000
12272 of 13000
12273 of 13000
12274 of 13000
12275 of 13000
12276 of 13000
12277 of 13000
12278 of 13000
12279 of 13000
12280 of 13000
12281 of 13000
12282 of 13000
12283 of 13000
12284 of 13000
12285 of 13000
12286 of 13000
12287 of 13000
12288 of 13000
12289 of 13000
12290 of 13000
12291 of 13000
12292 of 13000
12293 of 13000
12294 of 13000
12295 of 13000
12296 of 13000
12297 of 1

12789 of 13000
12790 of 13000
12791 of 13000
12792 of 13000
12793 of 13000
12794 of 13000
12795 of 13000
12796 of 13000
12797 of 13000
12798 of 13000
12799 of 13000
12800 of 13000
12801 of 13000
12802 of 13000
12803 of 13000
12804 of 13000
12805 of 13000
12806 of 13000
12807 of 13000
12808 of 13000
12809 of 13000
12810 of 13000
12811 of 13000
12812 of 13000
12813 of 13000
12814 of 13000
12815 of 13000
12816 of 13000
12817 of 13000
12818 of 13000
12819 of 13000
12820 of 13000
12821 of 13000
12822 of 13000
12823 of 13000
12824 of 13000
12825 of 13000
12826 of 13000
12827 of 13000
12828 of 13000
12829 of 13000
12830 of 13000
12831 of 13000
12832 of 13000
12833 of 13000
12834 of 13000
12835 of 13000
12836 of 13000
12837 of 13000
12838 of 13000
12839 of 13000
12840 of 13000
12841 of 13000
12842 of 13000
12843 of 13000
12844 of 13000
12845 of 13000
12846 of 13000
12847 of 13000
12848 of 13000
12849 of 13000
12850 of 13000
12851 of 13000
12852 of 13000
12853 of 13000
12854 of 13000
12855 of 1

In [4]:
images = np.asarray(images)       
len(images)  

13000

In [5]:
images[0]

array([[[ 72.],
        [ 78.],
        [ 81.],
        ...,
        [ 52.],
        [ 43.],
        [ 40.]],

       [[ 65.],
        [ 62.],
        [ 59.],
        ...,
        [ 55.],
        [ 52.],
        [ 45.]],

       [[ 47.],
        [ 42.],
        [ 60.],
        ...,
        [ 47.],
        [ 54.],
        [ 49.]],

       ...,

       [[ 91.],
        [ 65.],
        [ 46.],
        ...,
        [ 74.],
        [ 54.],
        [ 43.]],

       [[ 76.],
        [ 80.],
        [ 76.],
        ...,
        [108.],
        [ 69.],
        [ 46.]],

       [[ 77.],
        [ 74.],
        [ 81.],
        ...,
        [105.],
        [108.],
        [ 82.]]], dtype=float32)

In [6]:
# Get image size
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

[50 50  1]


In [7]:
df = pd.DataFrame(list(zip(targets, labels,images,file_paths)),columns=["emotions","labels","images","paths"])
df.head()

,emotions,labels,images,paths
0,angry,0,"[[[72.0], [78.0], [81.0], [75.0], [59.0], [54....",Data/angry/0.jpg
1,angry,0,"[[[152.0], [149.0], [147.0], [157.0], [146.0],...",Data/angry/1.jpg
2,angry,0,"[[[29.0], [25.0], [21.0], [23.0], [26.0], [24....",Data/angry/10.jpg
3,angry,0,"[[[32.0], [23.0], [20.0], [56.0], [43.0], [34....",Data/angry/10002.jpg
4,angry,0,"[[[222.0], [218.0], [202.0], [189.0], [199.0],...",Data/angry/10016.jpg


In [8]:
# Scale
images = images / 255


#Split into training and testing data
X = images
y = labels
from sklearn.model_selection import train_test_split
# 80/10/10 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.111, random_state=1)

X_train.shape

(10401, 50, 50, 1)

In [ ]:
X_val.shape

(1299, 50, 50, 1)

In [ ]:
X_test.shape

(1300, 50, 50, 1)

In [ ]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
from keras.utils import to_categorical
num_classes=7

# Prepare y data
y_binary_train = to_categorical(y_train, num_classes=num_classes, dtype='float32')
y_binary_test = to_categorical(y_test, num_classes=num_classes, dtype='float32')

In [ ]:
y_binary_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [ ]:
# Define hyperparamters
MIN_NEURONS = 20
MAX_NEURONS = 120
KERNEL = (3, 3)
n_layers = 4
num_features = 64
# Training hyperparamters
EPOCHS = 150
BATCH_SIZE = 500
PATIENCE = 10
width, height = 50, 50
# Determine the # of neurons in each convolutional layer
steps = np.floor(MAX_NEURONS / (n_layers + 1))
nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
nuerons = nuerons.astype(np.int32)

# Define a model
model = Sequential()

# Add convolutional layers
for i in range(0, n_layers):
    if i == 0:
        shape = (image_size[0], image_size[1], image_size[2])
        model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
    else:
        model.add(Conv2D(nuerons[i], KERNEL))

    model.add(Activation('relu'))

# Add max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(MAX_NEURONS))
model.add(Activation('relu'))

# Add output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
#desinging the CNN
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
# Print a summary of the model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 128)       512       
__________

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=2, mode='auto')

In [ ]:
# # TensorBoard callback
# LOG_DIRECTORY_ROOT = ''
# now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
# log_dir = "{}/run-{}/".format(LOG_DIRECTORY_ROOT, now)
# tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)

In [ ]:
# Place the callbacks in a list
callbacks = [early_stopping]

In [ ]:
# Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks, verbose=3)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
Epoch 2/150


In [ ]:
# Make a prediction on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions , axis=1)
test_predictions

In [ ]:
# Report the accuracy
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: " + str(accuracy))

In [ ]:
y_binary_test

In [ ]:
 # Save the model
# model.save("emotion_model_trained.h5")